In [1]:
import os
import json

from datasets import Dataset
from transformers import AutoTokenizer
from transformers import DefaultDataCollator
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
with open('dataset/dev-v1.1.json', 'r') as _f:
    dev = json.load(_f)

with open('dataset/train-v1.1.json', 'r') as _f:
    train = json.load(_f)

# SQuAD dataset

The Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset consisting of questions posed by crowdworkers on a set of Wikipedia articles. The answer to every question is a segment of text, or span, from the corresponding reading passage. There are 100,000+ question-answer pairs on 500+ articles [2].


SQuAD dataset holds the data inputs as bellow [1]:

![image](inputs/dataset_example.png)

as shown in the imgae above, Stanford's SQuAD dataset is dictionary with two keyes: 'data', and 'version'

In [94]:
type(train)

dict

In [95]:
len(train)

2

In [96]:
train['version']

'1.1'

The main part of this dataset is ```train['data']``` where questions and answers exist

```train['data']```  contains an array of instances, each belonging to a Wikipedia article with ```title``` and ```paragraphs```. 

In [73]:
# how many article are used in SQuAD datasets (for training part)?
len(train['data'])

442

In [75]:
# each article in this dataset contains 'title' and 'paragraphs'
train['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [82]:
# the first article is an article on 'University_of_Notre_Dame'
train['data'][0]["title"]

'University_of_Notre_Dame'

the most important part is the ```paragraph``` part since it contains 
```question```, ```answers``` in ```qas``` keys

In [83]:
type(train['data'][0]["paragraphs"])

list

In [85]:
# how many the first article has?
len(train['data'][0]["paragraphs"])

55

In [86]:
# what is the firs paragraph?
train['data'][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

As can be seen, each pagraph has a ```context``` and from that there are some questions asked and 
answered.

so now, it's read to read the dataset and store it as we want

In [90]:
def createDataset(inp_dict):
    contexts = []
    questions = []
    answers = []

    for article in inp_dict['data']:
 
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qs in paragraph['qas']:
                question = qs['question']
                for answer in qs['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append({'text': answer['text'], 'answer_start': answer['answer_start']})

    # build a dataset and return it as an object from Datset class. 
    return Dataset.from_dict({
    'context': contexts,
    'question': questions,
    'answers': answers
    })

In [91]:
TRAIN = createDataset(train)
TEST = createDataset(dev)

TRAIN = TRAIN.train_test_split(test_size=0.2, seed=1)

In [92]:
TRAIN

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 70079
    })
    test: Dataset({
        features: ['context', 'question', 'answers'],
        num_rows: 17520
    })
})

In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")
data_collator = DefaultDataCollator()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def preprocess_function(examples):
    
    questions = [q.strip() for q in examples["question"]]
    
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=500,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
train_tokenized = TRAIN.map(preprocess_function, batched=True, remove_columns=TEST.column_names)
test_tokenized = TEST.map(preprocess_function, batched=True, remove_columns=TEST.column_names)

Map:   0%|          | 0/70079 [00:00<?, ? examples/s]

Map:   0%|          | 0/17520 [00:00<?, ? examples/s]

Map:   0%|          | 0/34726 [00:00<?, ? examples/s]

In [10]:
training_args = TrainingArguments(
    output_dir="my_model",
    evaluation_strategy="epoch",
    save_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    report_to = 'none',
    load_best_model_at_end = True,
    overwrite_output_dir = True,
    metric_for_best_model= "eval_loss",
    greater_is_better= False
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized["train"],
    eval_dataset=train_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.394500,1.216926
2,1.106100,1.109576
3,0.940700,1.085470
4,0.819800,1.097360
5,0.707000,1.176460
6,0.619400,1.213709
7,0.531600,1.278332
8,0.478100,1.329728
9,0.442300,1.363711
10,0.416600,1.388544


TrainOutput(global_step=43800, training_loss=0.7902775059878554, metrics={'train_runtime': 17260.9226, 'train_samples_per_second': 40.6, 'train_steps_per_second': 2.538, 'total_flos': 8.941443942522e+16, 'train_loss': 0.7902775059878554, 'epoch': 10.0})

In [13]:
trainer.save_model('model/saved_model')

In [14]:
trainer.evaluate(eval_dataset=test_tokenized)

{'eval_loss': 1.233585000038147,
 'eval_runtime': 263.5852,
 'eval_samples_per_second': 131.745,
 'eval_steps_per_second': 8.236,
 'epoch': 10.0}

In [15]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="model/saved_model")

In [16]:
context = """India, a captivating mosaic of cultures, religions, and traditions, enthralls with its unparalleled diversity. Its ancient civilization, steeped in history and spanning millennia, has profoundly influenced global heritage. From the vibrant markets of Delhi to the idyllic shores of Goa, India's landscapes boast a breathtaking array of sights and experiences. Chennai, a city in the state of tamil nadu is expected to be the next tech hub in India following bangalore and mumbai. Yet, amidst the splendor, persistent challenges such as poverty and environmental degradation demand attention. With 28 states and 8 union territories, India stands as a testament to unity in diversity. In this vast expanse, Mumbai, the bustling metropolis pulsating with life, claims the title of the largest city by population. As India strides confidently into the future, it remains a land of contrasts, blending tradition with modernity, and embracing its complexities with resilience and grace."""

question_1 = 'Which city is the future tech hub?'
question_2 = 'Which city has the most population?'
question_3 = 'How many states and union territories are there in india?'

print(question_answerer(question=question_1, context=context))
print(question_answerer(question=question_2, context=context))
print(question_answerer(question=question_3, context=context))


{'score': 0.5948792695999146, 'start': 360, 'end': 367, 'answer': 'Chennai'}
{'score': 0.9208379983901978, 'start': 705, 'end': 711, 'answer': 'Mumbai'}
{'score': 0.4248407483100891, 'start': 597, 'end': 599, 'answer': '28'}


In [17]:
context = """ Following the great success of ChatGPT, there has been a proliferation of open-source large language models that are finetuned to follow instructions. These models are capable of providing valuable assistance in response to users’ questions/prompts. Notable examples include Alpaca and Vicuna, based on LLaMA, and OpenAssistant and Dolly, based on Pythia."""
question_1 = 'What happend after the success of ChatGPT?'

print(question_answerer(question=question_1, context=context))

{'score': 0.27822205424308777, 'start': 41, 'end': 108, 'answer': 'there has been a proliferation of open-source large language models'}


In [27]:
#!pip install pypdf

In [ ]:
# from pypdf import PdfReader 
  
# # creating a pdf reader object 
# reader = PdfReader('example.pdf') 

In [39]:
with open("context.txt", "r") as f:
    context = f.read()

In [40]:
context

"\n\nMatariki is a star cluster that appears in the early morning sky in New Zealand during the mid-winter months.\n\nThe star cluster is well known throughout the world and at different times of the year can be seen around the globe. It is one of the brightest clusters in the sky, containing hundreds of member stars.\n\nMatariki has different names around the world. In English, it is called by its ancient Greek name, Pleiades or the Seven Sisters. In Hawaiian it is Makali’i, ‘eyes of royalty', and in Japan it is Subaru, meaning ‘gathered together’.\nWhat is the significance of Matariki is that Matariki is known as the Māori New Year in Te Ao Māori (the Māori world view).\n\nClosely connected with the maramataka (the Māori lunar calendar), the reappearance of the Matariki stars in the early morning sky brings the past year to a close and marks the beginning of the new year.\n\nMātauranga Māori (ancestral knowledge and wisdom) is at the heart of celebrations of the Matariki public holid

In [41]:
# context = """ Following the great success of ChatGPT, there has been a proliferation of open-source large language models that are finetuned to follow instructions. These models are capable of providing valuable assistance in response to users’ questions/prompts. Notable examples include Alpaca and Vicuna, based on LLaMA, and OpenAssistant and Dolly, based on Pythia."""

while (True):
    input_text = input(">>")
    if input_text == "exit":
        break
    output_text = question_answerer(question=input_text, context=context)["answer"]
    print(output_text)

# question_1 = 'What happend after the success of ChatGPT?'



>> when Matariki happens?


mid-winter from late May to early July


>> can I see Matariki stars with naked eyes?


only nine are visible


KeyboardInterrupt: Interrupted by user

In [24]:
input_text = input(">>")

>> he


# Resources

In [93]:
# [1] https://www.researchgate.net/figure/An-example-of-the-SQuAD-dataset-Each-dataset-contains-an-array-of-instances-each_fig2_341852018

In [ ]:
# [2] https://www.kaggle.com/datasets/stanfordu/stanford-question-answering-dataset?resource=download